# Interactive Building Footprint Download
### Uses Leafmap library to allow the user to draw a polygon on the map and download building footprints within the area of interest from the BING repository

In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
import mercantile
from tqdm import tqdm
import os
import tempfile
import leafmap
import json

### Open a map canvas for drawing

In [2]:
# Create an interactive map
m = leafmap.Map()
m.add_basemap("HYBRID")
m.set_center(lon = -98.5795,  lat=39.828175, zoom = 4)
m

Map(center=[39.828175, -98.5795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

### Save the Area of Interest as a geojson file

In [5]:
m.save_draw_features("draw.geojson")

# Path to your GeoJSON file
geojson_file = "draw.geojson"

# Read the GeoJSON file
with open(geojson_file, "r") as f:
    geojson_data = json.load(f)

# Extract the geometry information
if geojson_data["type"] == "FeatureCollection":
    # Assume we want the first feature for simplicity
    feature = geojson_data["features"][0]  # Adjust index as needed
    geometry = feature["geometry"]
elif geojson_data["type"] == "Feature":
    geometry = geojson_data["geometry"]
else:
    raise ValueError("Unsupported GeoJSON type")

# Structure the result as requested
aoi_geom = {
    "coordinates": geometry["coordinates"],
    "type": geometry["type"]
}

# Store the results of interactive map work in aoi_geom dictionary
aoi_shape = shapely.geometry.shape(aoi_geom)
# Pull the min max value into bounds
minx, miny, maxx, maxy = aoi_shape.bounds

# Name the footprint file for later use
output_fn = "test.geojson"


### Determine the tiles that overlap the AOI

In [6]:
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(int(mercantile.quadkey(tile)))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

The input area spans 1 tiles: [23103311]


### Pull the buildings based on the AOI

In [7]:
# Read a CSV file from a URL that contains metadata for a global building dataset
df = pd.read_csv(
    "https://minedbuildings.z5.web.core.windows.net/global-buildings/dataset-links.csv"
)

# Initialize an index counter and an empty GeoDataFrame to store the combined results
idx = 0
combined_gdf = gpd.GeoDataFrame()

# Create a temporary directory to store downloaded GeoJSON files
with tempfile.TemporaryDirectory() as tmpdir:
    # List to store temporary filenames
    tmp_fns = []
    
    # Loop through a list of quad keys (tiles) to process each one
    for quad_key in tqdm(quad_keys):  # tqdm is used to show a progress bar
        # Filter rows in the dataset matching the current quad key
        rows = df[df["QuadKey"] == quad_key]
        
        # Case 1: Exactly one matching row (unique QuadKey in dataset)
        if rows.shape[0] == 1:
            url = rows.iloc[0]["Url"]  # Extract the URL of the GeoJSON file
            
            # Download and parse the GeoJSON file into a DataFrame
            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(shapely.geometry.shape)  # Convert geometry strings to shapes
            
            # Convert the DataFrame to a GeoDataFrame with WGS84 (EPSG:4326) coordinate reference system
            gdf = gpd.GeoDataFrame(df2, crs=4326)
            # Define a temporary filename to save the GeoJSON file
            fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            tmp_fns.append(fn)  # Add the filename to the list
            
            # Save the GeoDataFrame to a GeoJSON file if it doesn't already exist
            if not os.path.exists(fn):
                gdf.to_file(fn, driver="GeoJSON")
        
        # Case 2: Multiple rows found for the same QuadKey (duplicate entries in the dataset)
        elif rows.shape[0] > 1:
            print(f"Multiple rows found for QuadKey: {quad_key}")
            # Loop through each row to process each URL
            for i in range(rows.shape[0]):
                url = rows.iloc[i]["Url"]  # Extract the URL
                
                # Download and parse the GeoJSON file
                df2 = pd.read_json(url, lines=True)
                df2["geometry"] = df2["geometry"].apply(shapely.geometry.shape)  # Convert geometry strings to shapes
                
                # Convert to a GeoDataFrame
                gdf = gpd.GeoDataFrame(df2, crs=4326)
                # Define a unique filename for each duplicate entry
                fn = os.path.join(tmpdir, f"{quad_key}_{i}.geojson")
                tmp_fns.append(fn)  # Add the filename to the list
                
                # Save the GeoDataFrame to a GeoJSON file if it doesn't already exist
                if not os.path.exists(fn):
                    gdf.to_file(fn, driver="GeoJSON")
        
        # Case 3: No rows found for the current QuadKey (error handling)
        else:
            raise ValueError(f"QuadKey not found in dataset: {quad_key}")

    # Merge all downloaded GeoJSON files into a single GeoDataFrame
    for fn in tmp_fns:
        # Read each GeoJSON file into a GeoDataFrame
        gdf = gpd.read_file(fn)
        # Filter geometries that fall within the area of interest (AOI)
        gdf = gdf[gdf.geometry.within(aoi_shape)]
        # Assign unique IDs to each geometry, incrementing the index counter
        gdf['id'] = range(idx, idx + len(gdf))
        idx += len(gdf)  # Update the index counter
        # Append the filtered GeoDataFrame to the combined GeoDataFrame
        combined_gdf = pd.concat([combined_gdf, gdf], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.66s/it]


### Write the output to geojson file

In [8]:
combined_gdf = combined_gdf.to_crs('EPSG:4326')
combined_gdf.to_file(output_fn, driver='GeoJSON')

In [9]:
in_geojson = "test.geojson"
m.add_geojson(in_geojson, layer_name="Footprints")
m

Map(bottom=209980.0, center=[33.86157820443925, -101.55835298512457], controls=(ZoomControl(options=['position…